# <div align="center"><b> Trabajo Final - Visión por computadora 2  - CEIA </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomndado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al sigiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/github/brunomaso1/uba-ceia/blob/ceia-nlp/ceia-nlp/Desafio%201.ipynb">Desafio 1</a></font></small></em> </div> 

<!-- TODO: Arreglar el link -->


* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

<!-- Colab -->
<!-- <div align="center"><img src="https://drive.google.com/uc?export=view&id=1tUnWeK372o_yu0LxkMwtC4IR2jorDipN" width="600" alt="Figura 1: Detección de palmeras mediante modelos de aprendizaje profundo."></div> -->

<div align="center"><img src="./resources/portada.jpeg" width="600" alt="Figura 1: Detección de palmeras mediante modelos de aprendizaje profundo."></div>

<div align="center"><small><em>Figura 1: Detección de palmeras mediante modelos de aprendizaje profundo.</em></small></div>

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Trabajo final - VPC2 - FIUBA                                                                                                           |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Palm tree detector                                                                                                                     |
| **Integrantes** | - Bruno Masoller (brunomaso1@gmail.com) </br> - Simón Rodriguez (simon.andre.r@gmail.com)                                              |

</div>

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

Inicialmente, cargamos los datos desde Google Drive (si no está disponible el link, se puede desacargar desde Kaggle también -hay información de esto más adelante-):

In [ ]:
# TODO: Mejorar comentarios
# TODO: Poner funciones dentro de los tags correspondientes
# TODO: Refactoring de funciones
# TODO: Codigo repetido?
# TODO: Organizar importaciones

import sys
import os
import warnings
import zipfile
import torch
import torchvision
from torchvision.transforms import functional as F
from torchvision.transforms import v2 as T
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xml.etree.ElementTree as ET
import cv2
import fiftyone as fo
import shutil
import random
from PIL import Image
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision import tv_tensors
from torchinfo import summary
from torch.utils.data import DataLoader

DOWNLOAD_RESOURCES = False # Permite indicar si descargar el conjunto o no
RUTA_CARPETA_RESOURCES = './resources'
RUTA_ARCHIVO_REQUERIMIENTOS = os.path.join(RUTA_CARPETA_RESOURCES, 'requirements.txt')
RUTA_CARPETA_UTILS = os.path.join(RUTA_CARPETA_RESOURCES, 'utils')
if DOWNLOAD_RESOURCES:
    print("Descargando recursos...")
    %pip install gdown
    !gdown https://drive.google.com/drive/folders/1DFfgoGQ8-zLwQU85Uyq0X72GX4IITInV?usp=sharing --folder

# Verificamos si la carpeta ha sido descargada correctamente
if os.path.exists(RUTA_CARPETA_RESOURCES):
    print(f"La carpeta {RUTA_CARPETA_RESOURCES} fue encontrada.")

    # Verificamos si el archivo de requerimientos existe
    if os.path.exists(RUTA_ARCHIVO_REQUERIMIENTOS):
        print("Archivo de requerimientos encontrado, instalando dependencias...")
        %pip install -r {RUTA_ARCHIVO_REQUERIMIENTOS}
    else:
        warnings.warn(f"El archivo {RUTA_ARCHIVO_REQUERIMIENTOS} no fue encontrado. No se instalarán los requerimientos.")

    if os.path.exists(RUTA_CARPETA_UTILS):
        sys.path.append(os.path.abspath('./resources/utils'))
        print(f"Se agregó la ruta {RUTA_CARPETA_UTILS} al system path.")
    else:
        raise FileNotFoundError(f"La carpeta {RUTA_CARPETA_UTILS} no fue encontrada. Verifique la descarga.")
else:
    raise FileNotFoundError(f"La carpeta {RUTA_CARPETA_RESOURCES} no fue encontrada. Verifique la descarga.")

from engine import train_one_epoch, evaluate # type: ignore

<!-- TODO: arreglar esto -->
<code>
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")
</code>

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [ ]:
# Parámetros
DOWNLOAD_RESOURCES = False # Permite indicar si descargar el conjunto o no

VERBOSE = True # Muestra época a época la evolución
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
RANDOM_SEED = 42
EPOCHS = 1

PALMERAS_KAGGLE_DATASET = 'palmeras_kaggle'

# Rutas
RUTA_CARPETA_COMPRESSED = os.path.join(RUTA_CARPETA_RESOURCES, 'datasets', 'compressed')
RUTA_CARPETA_RAW = os.path.join(RUTA_CARPETA_RESOURCES, 'datasets', 'raw')
RUTA_CARPETA_KAGGLE_DATASET = os.path.join(RUTA_CARPETA_RAW, PALMERAS_KAGGLE_DATASET)

FORMATOS_IMAGENES = ('.jpg', '.jpeg', '.png')
FORMATOS_LABELS = ('.xml')
IMG_SIZE_W, IMG_SIZE_H = 1000, 1000

LABELS = {
    'Palm' : 1
}

TRAIN_TEST_SPLIT_RATIO = [0.8, 0.2]

random.seed(RANDOM_SEED)

print(f'Dispositivo actual: {DEVICE}')

## Introduction

En el marco de la especialización en Inteligencia Artificial de la Facultad de Ingeniería de la Universidad de Buenos Aires, se plantea, como trabajo final de la materia Visión por Computadora 2, desarrollar un modelo de visión por computadora que utilice aprendizaje profundo para detectar las ubicaciones de palmeras en diversas imágenes áereas.

Metodológicamente, se aplican principios de [CRISP-ML(Q)](https://ml-ops.org/content/crisp-ml) para el desarrollo del problema.

## Business and data understanding

La detección de palmeras es ampliamente estudiado en el área de visión por computadora, dado su potencial como insumo de otras soluciones, como por ejemplo, aquellas que intentan mejorar la salud de las palmeras, donde inicalmente es deseable una detección aera para luego realizar una inspección manual (con el objetivo de reducir costos).

Muchos estudios (como [este](https://www.researchgate.net/publication/382742087_Implementation_of_Slicing_Aided_Hyper_Inference_SAHI_in_YOLOv8_to_Counting_Oil_Palm_Trees_Using_High-Resolution_Aerial_Imagery_Data) por ejemplo) se realizan en paises en donde la mayoría de las palmeras son materia prima para la producción del "aceite de palmera". Sin embargo, en paises no productores, el cuidado de las palmeras también es importante, dado su impacto en el medio ambiente. 

## Data preparation

Los datos fueron obenidos del sito de Kaggle: https://www.kaggle.com/datasets/riotulab/aerial-images-of-palm-trees, sin embargo, para una mejor manejo, se los importan desde Google Drive (https://drive.google.com/drive/folders/1DFfgoGQ8-zLwQU85Uyq0X72GX4IITInV)

### Data visualization

Luego de descargado los datos, tenemos la siguiente estructura de directorios:

<!-- https://tree.nathanfriend.io/
- resources
  - datasets
    - compressed
      - palmeras_kaggle.zip
 -->

<pre>
resources/
└── datasets/
    └── compressed/
        └── palmeras_kaggle.zip
</pre>

Descomprimimos los datos:

In [3]:
# Función para descomprimir archivos .zip
def descomprimir_archivos(carpeta_origen, carpeta_destino):
    archivos = [f for f in os.listdir(carpeta_origen) if f.endswith('.zip')]
    
    if not archivos:
        warnings.warn("No se encontraron archivos .zip para descomprimir.")
        return
    
    for archivo in archivos:
        ruta_archivo = os.path.join(carpeta_origen, archivo)
        print(f"Descomprimiendo {archivo} en {carpeta_destino}...")
        
        # Descomprimir archivo
        with zipfile.ZipFile(ruta_archivo, 'r') as zip_ref:
            zip_ref.extractall(carpeta_destino)
        print(f"{archivo} descomprimido exitosamente.")

In [ ]:
# Crear la carpeta 'raw' si no existe
os.makedirs(RUTA_CARPETA_RAW, exist_ok=True)

# Verificar si la carpeta compressed existe
if os.path.exists(RUTA_CARPETA_COMPRESSED):
    descomprimir_archivos(RUTA_CARPETA_COMPRESSED, RUTA_CARPETA_RAW)
else:
    warnings.warn(f"La carpeta {RUTA_CARPETA_COMPRESSED} no fue encontrada. Verifique la estructura")

Una vez descomprimido los archivos, podemos observar que el conjunto está en formato PascalVOC (aunque también tiene dos archivos .csv que pueden utilizarse para otros formatos, ej: YOLO).

Ajustamos los archvios para tener una carpeta "data" y otra "labels", de la siguiente forma:

<!--
- Palm-Counting-349images/
  - data/
    - img1.jpg
    - img2.jpg
    - ...
  - labels/
    - img1.xml
    - img2.xml
    - ...
 -->

 <pre>
palmeras_kaggle/
├── data/
│   ├── img1.jpg
│   ├── img2.jpg
│   └── ...
└── labels/
    ├── img1.xml
    ├── img2.xml
    └── ...
 </pre>


In [5]:
# Función para mover archivos a la carpeta correspondiente
def mover_archivos(source_dir, data_dest, label_dest):
    for filename in os.listdir(source_dir):
        file_path = os.path.join(source_dir, filename)
        
        # Verifica si es un archivo de imagen (extensión .jpg o similar)
        if filename.endswith(FORMATOS_IMAGENES):
            shutil.move(file_path, os.path.join(data_dest, filename))
        # Verifica si es un archivo XML
        elif filename.endswith(FORMATOS_LABELS):
            shutil.move(file_path, os.path.join(label_dest, filename))

In [ ]:
# Define los paths de las carpetas
base_dir = os.path.join(RUTA_CARPETA_RAW, "Palm-Counting-349images")
train_dir = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, "train")
test_dir = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, "test")

# Renombrar la carpeta Palm-Counting-349images a palmeras_kaggle
if os.path.exists(base_dir):
    if not os.path.exists(RUTA_CARPETA_KAGGLE_DATASET):
        os.rename(base_dir, RUTA_CARPETA_KAGGLE_DATASET)

        # Define los nuevos directorios dentro de la nueva carpeta
        data_dir = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, "data")
        label_dir = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, "labels")

        # Crea las nuevas carpetas si no existen
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(label_dir, exist_ok=True)

        # Mover archivos de train y test a las nuevas carpetas
        try:
            mover_archivos(train_dir, data_dir, label_dir)
            mover_archivos(test_dir, data_dir, label_dir)
        except:
            warnings.warn(
                f"Error al mover los archivos. Las carpetas {train_dir} y {test_dir} existen? O ya fueron borradas?"
            )

        # Eliminar las carpetas de train y test
        if os.path.exists(train_dir):
            shutil.rmtree(train_dir)
        if os.path.exists(test_dir):
            shutil.rmtree(test_dir)

        # Elimina los archivos .csv
        csv_files = ["train_labels.csv", "test_labels.csv"]
        for csv_file in csv_files:
            csv_path = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, csv_file)
            if os.path.exists(csv_path):
                os.remove(csv_path)

        print(
            "Archivos movidos, archivos CSV eliminados y la carpeta renombrada correctamente."
        )
    else:
        warnings.warn(
            f"Ya existe la ruta {RUTA_CARPETA_KAGGLE_DATASET}, no se hizo nada..."
        )
else:
    raise FileNotFoundError(f'No se descomprimió el dataset {base_dir}')

Visualizamos la cantidad de imágenes:

In [ ]:
# Función para contar archivos .jpg en una carpeta dada
def contar_imagenes(directorio):
    return len([f for f in os.listdir(directorio) if f.endswith(FORMATOS_IMAGENES)])

# Contar imágenes en las carpetas 'test' y 'train'
cant_imagenes = contar_imagenes(os.path.join(RUTA_CARPETA_KAGGLE_DATASET, 'data'))

print(f'Total de imágenes: {cant_imagenes}')

Si inspeccionamos algunas imágenes, vemos que tienen un tag `polygon` en el xml. Este tag lo vamos a eliminar para mejor manejo:

In [8]:
# TODO: eliminar tag path
# Función para eliminar los objetos con polygon
def remove_objects_with_polygon(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Encontrar todos los objetos
    for obj in root.findall("object"):
        # Si el objeto tiene un tag polygon, eliminar el objeto
        if obj.find("polygon") is not None:
            root.remove(obj)

    # Guardar los cambios en el archivo
    tree.write(xml_path)

In [ ]:
# Directorio de etiquetas (carpeta que contiene los archivos XML)
label_dir = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, "labels")

# Recorrer todos los archivos XML en la carpeta de etiquetas
for xml_file in os.listdir(label_dir):
    if xml_file.endswith(".xml"):
        xml_file_path = os.path.join(label_dir, xml_file)
        # Aplicar la función a cada archivo XML
        remove_objects_with_polygon(xml_file_path)
        print(f"Se han eliminado los objetos con polygon de {xml_file_path}")


print("Proceso completado para todos los archivos.")


Podemos observar una imagen de ejemplo y su detección:

In [10]:
# TODO: Mejorar las etiquetas y visualizacion.
# Función para dibujar bounding boxes, ya sea desde archivos o dataset
def draw_bbox(img, target_or_annotation_path, ax, from_dataset=False):
    if from_dataset:
        # Si es desde el dataset, el formato de 'img' ya es un tensor
        img_np = img.permute(1, 2, 0).numpy()  # Convertir a formato de Matplotlib
        ax.imshow(img_np)
        ax.axis("off")

        # Dibujar los bounding boxes del dataset
        for box in target_or_annotation_path["boxes"]:
            xmin, ymin, xmax, ymax = box
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                     linewidth=2, edgecolor="r", facecolor="none")
            ax.add_patch(rect)

        # Añadir etiquetas
        for box, label in zip(target_or_annotation_path["boxes"], target_or_annotation_path["labels"]):
            xmin, ymin, _, _ = box
            ax.text(xmin, ymin - 10, "Palmera", color="white", fontsize=12, backgroundcolor="red")
    else:
        # Si es desde archivos, cargar la imagen y parsear el archivo XML
        image = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        ax.axis("off")

        # Parsear el archivo de anotación PascalVOC
        tree = ET.parse(target_or_annotation_path)
        root = tree.getroot()
        for obj in root.findall("object"):
            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            # Dibujar el bounding box
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                     linewidth=2, edgecolor="r", facecolor="none")
            ax.add_patch(rect)

            # Mostrar etiqueta
            label = obj.find("name").text
            ax.text(xmin, ymin - 10, label, color="white", fontsize=12, backgroundcolor="red")

# Función para dibujar una grilla de imágenes con bounding boxes
def draw_grid(images, targets, grid_size=(3, 3), from_dataset=False):
    fig, axs = plt.subplots(grid_size[0], grid_size[1], figsize=(15, 15))

    for i in range(grid_size[0] * grid_size[1]):
        img = images[i]
        target = targets[i]
        ax = axs[i // grid_size[1], i % grid_size[1]]
        draw_bbox(img, target, ax, from_dataset=from_dataset)

    # Ajustar el layout para evitar superposición
    plt.tight_layout()
    plt.show()

# Función para mostrar una sola imagen
def show_single_image(image, target, from_dataset=False):
    fig, ax = plt.subplots(figsize=(8, 8))
    draw_bbox(image, target, ax, from_dataset=from_dataset)
    plt.tight_layout()
    plt.show()


# Función para obtener las rutas de imágenes y anotaciones
def get_image_annotation_paths(image_dir, label_dir, max_files=9):
    # Listar todos los archivos de imágenes (filtrar por extensión de imagen)
    image_files = [f for f in os.listdir(image_dir) if f.endswith(FORMATOS_IMAGENES)]
    # Ordenar los archivos para garantizar el orden
    image_files.sort()

    # Tomar las primeras 'max_files' imágenes
    image_files = image_files[:max_files]

    # Generar las rutas para las imágenes y las anotaciones XML correspondientes
    image_paths = [os.path.join(image_dir, img) for img in image_files]

    # Reemplazar la extensión de la imagen por .xml para encontrar las anotaciones
    annotation_paths = [
        os.path.join(label_dir, os.path.splitext(img)[0] + ".xml")
        for img in image_files
    ]

    return image_paths, annotation_paths

# Función para obtener una imagen y su anotación aleatoria
def get_random_image_and_annotation(image_dir, label_dir):
    # Listar todas las imágenes disponibles en la carpeta
    image_files = [f for f in os.listdir(image_dir) if f.endswith(FORMATOS_IMAGENES)]
    
    # Seleccionar una imagen aleatoria
    random_image = random.choice(image_files)
    
    # Generar las rutas para la imagen y la anotación correspondiente
    image_path = os.path.join(image_dir, random_image)
    annotation_path = os.path.join(label_dir, os.path.splitext(random_image)[0] + ".xml")
    
    return image_path, annotation_path

In [ ]:
image_dir = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, "data")
label_dir = os.path.join(RUTA_CARPETA_KAGGLE_DATASET, "labels")

image_path, annotation_path = get_random_image_and_annotation(image_dir, label_dir)
show_single_image(image_path, annotation_path, from_dataset=False)

También visualizamos varias imágenes:

In [ ]:
image_paths, annotation_paths = get_image_annotation_paths(image_dir, label_dir, max_files=9)
draw_grid(image_paths, annotation_paths, grid_size=(3, 3), from_dataset=False)

También podemos utilizar la librería de FiftyOne (https://docs.voxel51.com/) para una mejor visualización. Otra ventaja de esta librería es que tiene un módulo de exportación a varios formatos conocidos:

In [ ]:
# TODO: Investigar más esto.
# TODO: Arreglar annotaciones, es posible con esto?
# Crear el dataset en FiftyOne desde un formato PascalVOC
fiftyone_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.VOCDetectionDataset,
    dataset_dir=RUTA_CARPETA_KAGGLE_DATASET
)

Podemos observar los datos del conjunto:

In [ ]:
print(fiftyone_dataset)

In [ ]:
# TODO: Mejorar esta imagen, está cargando antes de tiempo... Poner un sleep?
# Lanzar la aplicación FiftyOne desde Jupyter
session = fo.launch_app(fiftyone_dataset)

session.freeze()
session.open_tab()

<!-- TODO: Balance de clases? Análisis de datos? -->

## Machine learning model engineering

Como experimentos, dado que inicialmente el modelo no tiene como objetivo el funcionamiento en tiempo real, se plantea utilizar modelos de detección en dos etapas (ej: Faster R-CNN). Sin embargo, también se tienen en cuenta modelos de una sola etapa (ej: YOLO)

### Faster R-CNN

#### Preparar el datset

En el formato actual del conjunto, es necesario generar una clase Dataset Customizada para el entrenamiento. Para eso podemos tomar como ejemplo este tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

📝 <em><font color='Gray'>Nota:</font></em>
<em><font color='Gray'><p>

Para entrenar la red Faster R-CNN, es necesario que se cumplan algunos requerimientos, entre los cuales se encuentra una definición de la estructura del Dataset como la siguiente:

<cite>
The only specificity that we require is that the dataset __getitem__ should return a tuple:

- image: torchvision.tv_tensors.Image of shape [3, H, W], a pure tensor, or a PIL Image of size (H, W)
- target: a dict containing the following fields
    - `boxes`, torchvision.tv_tensors.BoundingBoxes of shape [N, 4]: the coordinates of the N bounding boxes in [x0, y0, x1, y1] format, ranging from 0 to W and 0 to H
    - `labels`, integer torch.Tensor of shape [N]: the label for each bounding box. 0 represents always the background class.
    - `image_id`, int: an image identifier. It should be unique between all the images in the dataset, and is used during evaluation
    - `area`, float torch.Tensor of shape [N]: the area of the bounding box. This is used during evaluation with the COCO metric, to separate the metric scores between small, medium and large boxes.
    - `iscrowd`, uint8 torch.Tensor of shape [N]: instances with iscrowd=True will be ignored during evaluation.
    - (optionally) `masks`, torchvision.tv_tensors.Mask of shape [N, H, W]: the segmentation masks for each one of the objects

</cite>

También destacar que el modelo considera a la clase 0 como el fondo, lo que implica consideración en las labels

</p></font></em>

In [16]:
class PalmTreeDataset(Dataset):
    def __init__(self, root):
        self.root = root
        self.imgs = list(sorted(os.listdir(os.path.join(root, "data"))))
        self.labels = list(sorted(os.listdir(os.path.join(root, "labels"))))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        # Cargar imagen
        img_path = os.path.join(self.root, "data", self.imgs[idx])
        label_path = os.path.join(self.root, "labels", self.labels[idx])
        img = read_image(img_path)

        # Parsear archivo XML (PascalVOC)
        tree = ET.parse(label_path)
        root = tree.getroot()
        boxes = []
        labels = []
        isCrowd = []
        for obj in root.findall('object'):
            label = obj.find('name').text
            if label in LABELS.keys():
                bbox = obj.find('bndbox')
                xmin = int(bbox.find('xmin').text)
                ymin = int(bbox.find('ymin').text)
                xmax = int(bbox.find('xmax').text)
                ymax = int(bbox.find('ymax').text)

                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(LABELS[label]) # Hay solo una clase: Palmera
                isCrowd.append(0) # Seteamos como falso.

        # Convertimos a tensores
        isCrowd = torch.as_tensor(isCrowd, dtype=torch.int64)        
        labels = torch.as_tensor(labels, dtype=torch.int64)
        img = tv_tensors.Image(img)
        boxes = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_image_size(img))

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = isCrowd

        return img, target

In [17]:
class TransformedPalmDataset(Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, target = self.dataset[idx]
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target

#### Definir las transformaciones

In [18]:
def get_transforms(train):
    transforms = []
    if train:
        # TODO: Train augmentantions
        pass
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

Creamos los datasets:

In [19]:
full_dataset = PalmTreeDataset(RUTA_CARPETA_KAGGLE_DATASET)
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, TRAIN_TEST_SPLIT_RATIO, torch.Generator().manual_seed(RANDOM_SEED))

train_dataset = TransformedPalmDataset(train_dataset, transforms=get_transforms(train=True))
test_dataset = TransformedPalmDataset(test_dataset, transforms=get_transforms(train=False))

Visualizamos las transformaciones:

In [ ]:
draw_grid(
    [train_dataset[i][0] for i in range(9)],
    [train_dataset[i][1] for i in range(9)],
    grid_size=(3, 3),
    from_dataset=True,
)

#### Definición del modelo

In [21]:
def get_model(num_classes):
    # Cargar un modelo preentrenado de Faster R-CNN
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights='DEFAULT')
    
    # Obtener el número de entradas de la capa de clasificación
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Reemplazar la cabeza del clasificador por una que tenga el número de clases deseado (n clase + fondo)
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes + 1)
    
    return model

In [ ]:
# Instanciamos el modelo.
model = get_model(len(LABELS.keys()))

print(model)

In [ ]:
img, _ = train_dataset[0]
input_size = (1, *img.shape)
summary(model, input_size=input_size)

Testeamos la inferencia:

In [ ]:
data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x))
)

# Training
images, targets = next(iter(data_loader))
output = model(images, targets)  # Returns losses and detections
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
print(f"Training test: {output}")

# Inferencia
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(f"Inference test: {predictions[0]}")

#### Preparar el entrenamiento

In [25]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Enviar modelo a GPU si está disponible
model.to(DEVICE)

# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Aprender tasa de aprendizaje (LR) programada
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


#### Entrenar el modelo

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train_one_epoch(model, optimizer, train_dataloader, DEVICE, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, test_dataloader, DEVICE)

### YOLO

<!-- ## Quality assurance for machine learning applications -->

<!-- ## Deployment -->

<!-- ## Monitoring and maintenance -->

## Referencias

- Adel Ammar, and Anis Koubaa. (2023). Aerial images of palm trees [Data set]. Kaggle. https://doi.org/10.34740/KAGGLE/DSV/6382990
- TorchVision Object Detection Finetuning Tutorial — PyTorch Tutorials 2.4.0+cu121 documentation. (n.d.). https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html